<a href="https://colab.research.google.com/github/noor-osama/facePosingPrediction/blob/main/headPoseEstimation_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/Database/AFLW2000-3D.zip
!unzip AFLW2000-3D.zip

--2022-06-03 14:12:32--  http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/Database/AFLW2000-3D.zip
Resolving www.cbsr.ia.ac.cn (www.cbsr.ia.ac.cn)... 159.226.21.60
Connecting to www.cbsr.ia.ac.cn (www.cbsr.ia.ac.cn)|159.226.21.60|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87413900 (83M) [application/x-zip-compressed]
Saving to: ‘AFLW2000-3D.zip’

AFLW2000-3D.zip     100%[===================>]  83.36M   324KB/s    in 4m 24s  

2022-06-03 14:16:58 (323 KB/s) - ‘AFLW2000-3D.zip’ saved [87413900/87413900]

Archive:  AFLW2000-3D.zip
   creating: AFLW2000/
   creating: AFLW2000/Code/
  inflating: AFLW2000/Code/DrawSolidHead.m  
  inflating: AFLW2000/Code/DrawTextureHead.m  
  inflating: AFLW2000/Code/main_show_without_BFM.m  
  inflating: AFLW2000/Code/main_show_with_BFM.m  
   creating: AFLW2000/Code/Mex/
  inflating: AFLW2000/Code/Mex/compile.m  
  inflating: AFLW2000/Code/Mex/Tnorm_Vnorm.h  
  inflating: AFLW2000/Code/Mex/Tnorm_VnormC.cpp  
  inflatin

In [2]:
import numpy as np
import os
import scipy.io as sio
import cv2
import math
from math import cos, sin
import dlib
import glob
from pathlib import Path
import pandas as pd
from PIL import Image, ImageFilter

In [3]:
mylist = [Path(f).stem for f in glob.glob("AFLW2000/*.mat")]
sio.loadmat(os.path.join('AFLW2000/',mylist[0]))

{'Color_Para': array([[ 0.8440023 ,  1.0719812 ,  1.2737658 , -0.11010647, -0.07766   ,
         -0.04330396,  1.        ]], dtype=float32),
 'Exp_Para': array([[-3.95290757],
        [-1.77342776],
        [-0.94872395],
        [ 0.87078476],
        [ 0.17700903],
        [ 0.0410185 ],
        [-0.79737847],
        [ 1.71817927],
        [ 0.16897713],
        [ 0.17156599],
        [ 0.66140333],
        [ 0.9052491 ],
        [ 0.32678599],
        [-1.20348187],
        [ 0.08650002],
        [-0.30113656],
        [ 0.02610228],
        [ 0.17025053],
        [-0.14090874],
        [ 0.37761379],
        [ 0.12873234],
        [-0.17973414],
        [ 0.06987645],
        [ 0.03926005],
        [ 0.57986301],
        [-0.25389097],
        [-0.0368891 ],
        [ 0.23785595],
        [-0.07355066]]),
 'Illum_Para': array([[3.22478592e-01, 2.85804391e-01, 2.30997548e-01, 1.05171955e+00,
         1.11781406e+00, 1.18432403e+00, 5.05438685e-01, 1.64901865e+00,
         2.1423595

In [8]:
def get_list_from_filenames(file_path):
    # input:    relative path to .txt file with file names
    # output:   list of relative path names
    lines = [Path(f).stem for f in glob.glob(file_path)]
    return lines

In [4]:
def get_pt3d_from_mat(mat_path):
    # Get 2D landmarks
    mat = sio.loadmat(mat_path)
    pt3d = mat['pt3d_68']
    return pt3d

In [5]:
def get_ypr_from_mat(mat_path):
    # Get yaw, pitch, roll from .mat annotation.
    # They are in radians
    mat = sio.loadmat(mat_path)
    # [pitch yaw roll tdx tdy tdz scale_factor]
    pre_pose_params = mat['Pose_Para'][0]
    # Get [pitch, yaw, roll]
    pose_params = pre_pose_params[:3]
    return pose_params

In [9]:
#prepare data

xs = ['x'+str(i) for i in range(1,69)]
ys = ['y'+str(i) for i in range(1,69)]
labels = ['pitch', 'yaw', 'roll']

column_names = xs + ys + labels
def get_data ():
  files = get_list_from_filenames("AFLW2000/*.mat")
  final = pd.DataFrame() 
  for file in files :
        d = get_pt3d_from_mat('AFLW2000/' + file)
        data = np.array(d, dtype=np.float32)
        #print(data.shape)
        df1= pd.DataFrame(data)
        df1=df1.iloc[:-1,:]
        #df1
        dfx = df1.iloc[:-1,:]
        #dfx
        dfy = pd.DataFrame(np.array(df1.iloc[-1,:]).reshape(1,-1))
        #dfy
        xy =  pd.DataFrame(pd.concat([dfx, dfy], join='inner', axis=1))
        pyr=pd.DataFrame(get_ypr_from_mat('AFLW2000/' + file).reshape(1,-1))
        #pyr
        result = pd.DataFrame(pd.concat([xy, pyr], join='inner', axis=1))
        final = final.append(result, ignore_index=True)


  final.columns = column_names
  return final 


In [10]:
#prepare dataframe 
df=get_data()
y = pd.DataFrame()
X = df.iloc[:,:-3]
pitch =pd.DataFrame(df.iloc[:,-3])
yaw =pd.DataFrame(df.iloc[:,-2])
roll =pd.DataFrame(df.iloc[:,-1])
